In [1]:
#Bunch of Imports
import pandas as pd
import numpy as np
import nltk
import re
import string
from pandas.io.json import json_normalize
import json
import glob
from langdetect import detect
import itertools as it
from gensim.models.wrappers import FastText

ImportError: No module named gensim.models.wrappers

# Read Data and remove Duplicates

In [2]:
#Regex init
url_regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
tokenizer = nltk.tokenize.TweetTokenizer()
lemmatizer = nltk.WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words('english'))
printable_chars = set(string.printable)
punctuations = set(string.punctuation)

In [3]:
'''
from langdetect.detector_factory import DetectorFactory
factory = DetectorFactory()
factory.load_profile('/Users/apple/Desktop/swisher/analysis/langdetect/profiles/')

def detect_lang(text):
    detector = factory.create()
    detector.append(text)
    return detector.detect()
'''

In [4]:
def filter_non_english_tweets(df):
    accepted_langs = ['en', 'et', 'ro', 'ca', 'sw', 'fi', 'sk', 'so', 'id', 'fr','pl', 'sv', 'hr', 'sl', 'tl', 'it', 'cy', 'sq', 'cs', 'hu']
    lang_list = []
    lang = ""
    for i in list(df['text']):
        try:
            lang = detect(i) 
            lang = "Not accepted" if lang not in accepted_langs else lang
        except Exception as e:
            pass
        lang_list.append(lang)
    df['lang'] = lang_list
    #df = df[df['lang'] != False]
    return df

In [5]:
def normalize_tweet(tweet,lemmatize=False):
    words = []
    clean_tweet = ''.join(filter(lambda c: c in printable_chars, tweet))
    for token, tag in nltk.pos_tag(tokenizer.tokenize(clean_tweet.lower())):
        if len(token) < 2 or url_regex.search(token):
            continue
        elif all(char in punctuations for char in token):
            continue
        elif token[0] == '@':
            token = '@person' #Replace all friend tags with a common token.
            tag = 'NNP'
        words.append(token)
    return words

In [6]:
normalize_tweet("ww Bro # I love swisher GAHDAMN @me", lemmatize=False)

[u'ww', u'bro', u'love', u'swisher', u'gahdamn', '@person']

In [7]:
def read_data(file_name):
    '''
        Read the data and convert createdAt to timestamp
        Drop cols=id, createdAt, latitude, longitude, place_country, place_name, place_type
    '''
    header = pd.read_csv(file_name, nrows=0)
    df = pd.read_csv(file_name,usecols=header)
    df['timestamp'] = pd.to_datetime(df['createdAt'],errors='coerce')
    
    
    
    df = df[df.isRetweet == '0']
    
    df = df.drop(['createdAt','isRetweet','latitude','longitude', 'place_country', 'place_name', 'place_type'],axis=1)
    df['id']=pd.to_numeric(df['id'], errors='coerce')
    df.dropna(subset=['id'],inplace=True)
    df['NormalizedText'] = list(map(normalize_tweet, df['text']))
    df = filter_non_english_tweets(df)
    return df

# Process the data

file_list = glob.glob("/Users/apple/Desktop/swisher/data/*.csv")
for fname in file_list:
    print "Processing:  ",fname,
    df = read_data(fname)
    print "Processed:  ",fname, "*** Count: ",len(df)
    
    df.to_csv('/Users/apple/Desktop/swisher/unique_tweets/'+fname.split('/')[-1], index=False)

# Read unique_tweets

In [10]:
data_df = pd.DataFrame()
file_list = glob.glob("/Users/apple/Desktop/swisher/unique_tweets/*.csv")
#file_list = ["/Users/apple/Desktop/swisher/unique_tweets/Nov01ToNov15.csv"]
for fname in file_list:
    temp = pd.read_csv(fname)
    data_df = data_df.append(temp)
    

In [11]:
df.head()

,id,text,userId,timestamp,NormalizedText,lang
0,1.041221e+18,Bro I love swisher GAHDAMN,3178697646,2018-09-16 00:02:05,"[bro, love, swisher, gahdamn]",en
1,1.041221e+18,@atendyne Its either a reg swisher or sweet ba...,2159965242,2018-09-16 00:03:01,"[@person, its, either, reg, swisher, or, sweet...",en
3,1.041224e+18,i just got a homie high for his first time eve...,852689860871602177,2018-09-16 00:14:09,"[just, got, homie, high, for, his, first, time...",en
6,1.041229e+18,"Ain't get to roll no weed, ain't get to roll n...",708542839,2018-09-16 00:36:32,"[ain't, get, to, roll, no, weed, ain't, get, t...",en
7,1.041231e+18,"I would make him look through the trash too, d...",3001624555,2018-09-16 00:43:06,"[would, make, him, look, through, the, trash, ...",en


In [17]:
df = data_df[~data_df['lang'].isnull()]
df = df[df['lang']!='Not accepted']
#data_df[~((data_df['lang'].isnull())(data_df['lang']=='Not accepted')  | (data_df['lang']=='NaN'))]

In [18]:
df.lang.unique()

array(['en', 'so', 'sv', 'sw', 'cy', 'sq', 'pl', 'tl', 'hu', 'ro', 'et',
       'hr', 'id', 'fr', 'it', 'ca', 'sl', 'fi', 'cs', 'sk'], dtype=object)

In [21]:
df.head()

,id,text,userId,timestamp,NormalizedText,lang
1,1.05789025932e+18,Social platforms operate with sloppy disregard...,19757554,2018-11-01 00:01:07,"[u'social', u'platforms', u'operate', u'with',...",en
2,1.05789072646e+18,Social platforms operate with sloppy disregard...,27365543,2018-11-01 00:02:58,"[u'social', u'platforms', u'operate', u'with',...",en
3,1.05789099743e+18,Social platforms operate with sloppy disregard...,2985579963,2018-11-01 00:04:03,"[u'social', u'platforms', u'operate', u'with',...",en
4,1.05789113735e+18,#ICanRelate\n#MustLoveCats\n\nTony Swisher htt...,27611383,2018-11-01 00:04:36,"[u'#icanrelate', u'#mustlovecats', u'tony', u'...",en
5,1.05789156982e+18,Social platforms operate with sloppy disregard...,3309380118,2018-11-01 00:06:19,"[u'social', u'platforms', u'operate', u'with',...",en


In [22]:
unique_userid = df['userId'].unique()
len(unique_userid)

71383

In [23]:
with open('user_id.csv', 'w') as f:
    for id in unique_userid:
        f.write("%s\n" % id)

In [24]:
len(unique_userid)

71383

# Process Bot users

In [26]:
bot_df['user_id']= user_id
bot_df['universal_score']=universal_score
bot_df['english_score'] = english_score

In [27]:
bot_df.head()

,user_id,universal_score,english_score
0,545455886,0.052029,0.053331
1,877167468,0.056245,0.067835
2,1042179458302717952,0.095903,0.085924
3,1001273707493380097,0.474489,0.180767
4,3725024597,0.138026,0.108277


In [28]:
len(bot_df)
bot_df.to_csv('bot_results.csv',index=False)

In [29]:
bot_df.describe()

,universal_score,english_score
count,40351.000000,40351.000000
mean,0.140096,0.130846
std,0.190728,0.205292
min,0.018465,0.015423
25%,0.037980,0.027678
50%,0.060780,0.041790
75%,0.128516,0.108277
max,0.986118,0.987588


# Filter data from Bots

In [69]:
bot_df= pd.read_csv('bot_results.csv')

In [70]:
bot_df.head()

,user_id,universal_score,english_score
0,545455886,0.052029,0.053331
1,877167468,0.056245,0.067835
2,1042179458302717952,0.095903,0.085924
3,1001273707493380097,0.474489,0.180767
4,3725024597,0.138026,0.108277


In [71]:
bot_threshold = 0.8

In [72]:
bot_df = bot_df[bot_df['universal_score']<=bot_threshold]


In [73]:
bot_df = bot_df[bot_df['english_score']<=bot_threshold]

In [74]:
len(df),len(bot_df)

(100270, 38887)

In [75]:
df_non_bots = df[df['userId'].isin(bot_df['user_id'])]

In [76]:
len(df_non_bots)

54835

In [77]:
df_non_bots.head()

,id,text,userId,timestamp,NormalizedText,lang
0,1.06334e+18,"Facebook and the Fires"" by KARA SWISHER via NY...",3057178511,2018-11-16 00:01:34,"[u'facebook', u'and', u'the', u'fires', u'by',...",en
2,1.06334e+18,"Facebook and the Fires"" by KARA SWISHER #Opini...",3057178511,2018-11-16 00:03:39,"[u'facebook', u'and', u'the', u'fires', u'by',...",en
3,1.06334e+18,@HoldMyPurseGirl That nasty ass swisher got me...,46276589,2018-11-16 00:14:33,"['@person', u'that', u'nasty', u'ass', u'swish...",en
4,1.06335e+18,"Facebook and the Fires"" Kara Swisher on why F...",51685865,2018-11-16 00:16:23,"[u'facebook', u'and', u'the', u'fires', u'kara...",en
5,1.06335e+18,I’m really a swisher man,40290165,2018-11-16 00:21:41,"[u'im', u'really', u'swisher', u'man']",en


In [78]:
df_non_bots.to_csv('data_non_bots.csv',index=False)

In [89]:
df_non_bots.groupby('userId').count().reset_index().sort_values(['text'],ascending=False).head()

,userId,id,text,timestamp,NormalizedText,lang
20884,2213693318,426,426,426,426,426
10709,338299487,210,210,210,210,210
26327,3242522096,203,203,203,203,203
2361,34923805,171,171,171,171,171
26325,3242429862,142,142,142,142,142


In [95]:
def to_gram(tweet_df):
    
    # Load the word vectors for the tweets 1-grams
    onegram_df = pd.DataFrame(list(nltk.FreqDist(it.chain(*tweet_df.NormalizedText)).items()),
                 columns=['Word', 'Freq'])
    onegram_df['Tweets'] = onegram_df.Word.apply(lambda word: set())
    onegram_df['TweetsCount'] = onegram_df.Word.apply(lambda word: 0)
    onegram_df['IsInteresting'] = onegram_df.Word.apply(lambda word: word not in stopwords and not word.strip().isdigit())
    print('Loading FastText model. Will take a few minutes...')
    ft_model = FastText.load_fasttext_format(FAST_TEXT_MODEL_FILE)
    print('FastText model loaded.')
    onegram_df['Vector'] = list(map(lambda x: ft_model[x] if x in ft_model else np.zeros(300), onegram_df.Word))
    onegram_df.set_index('Word', inplace=True)
    onegram_df.sort_values(['Freq'], ascending=[False], inplace=True)
    for i, tweet in enumerate(tweet_df.NormalizedText):
        for word in tweet:
            onegram_df.loc[word].Tweets.add(i)
            onegram_df.at[word, 'TweetsCount'] = len(onegram_df.loc[word].Tweets)
        
    # Load the bam histogram
    print('Processing bi-grams')
    bigram_df = pd.DataFrame(list(nltk.FreqDist(it.chain(*map(lambda x: map('-'.join,
                        filter(lambda bi: all(word not in stopwords and not word.strip().isdigit() for word in bi),
                               nltk.bigrams(x))), tweet_df.NormalizedText))).items()),
                columns=['Word', 'Freq'])
    bigram_df['Tweets'] = bigram_df.Word.apply(lambda word: set())
    bigram_df['TweetsCount'] = bigram_df.Word.apply(lambda word: 0)
    bigram_df.set_index('Word', inplace=True)
    bigram_df.sort_values(['Freq'], ascending=[False], inplace=True)
    for i, tweet in enumerate(tweet_df.NormalizedText):
        for bigram in filter(lambda bi: all(word not in stopwords and not word.strip().isdigit() for word in bi), nltk.bigrams(tweet)):
            key = '-'.join(bigram)
            bigram_df.loc[key].Tweets.add(i)
            bigram_df.at[key, 'TweetsCount'] = len(bigram_df.loc[key].Tweets)
    return tweet_df, onegram_df, bigram_df

In [96]:
meh = to_gram(df_non_bots)

Loading FastText model. Will take a few minutes...


NameError: global name 'FastText' is not defined